### This program will format the Contract Pipeline Report for Morgan Stanley. 

### This program does the following:
   #### 1. Reads the contract pipeline report into pandas
   #### 2. Isolates contracts in draft or draft amendment into df2
   #### 3. Filters on contracts expiring within 365 days and writes to df3
   #### 4. Combines df2 and df3 into 1 report
   #### 5. Reads a report of LOB and Legal Team reps into df_legal and df_biz
   #### 6. Drops duplicates
   #### 7. Renames the Project Id columns of all 3 dfs (contract_pipeline, df_legal and df_biz) to 'Project Id'
   #### 8. Joins all three dfs into one report (pipeline_final) on the 'Project Id' column
   #### 9. Removes duplicates again then writes the final to a CSV on the O:Drive

In [ ]:
#import required libraries

import pandas as pd
import numpy as np

In [ ]:
#read the pipeline report into pandas and remove all 'Unclassified'

df = pd.read_csv(r"filepath")
print("Pipeline Report read into dataframe")
df.replace("Unclassified", np.nan, inplace=True)

df.columns

In [ ]:
# create a dataframe, df2, of contracts in draft or draft amendment status from df

df2 = df[df['[PCW] Contract Status'].isin(["Draft", "Draft Amendment"])]

print(len(df2))
df2

In [ ]:
# check data types, date fields will need to be converted into dates
df.dtypes

In [ ]:
# convert dates to date types

df['[PCW]Contract (Expiration Date)']= pd.to_datetime(df['[PCW]Contract (Expiration Date)'])
df['[PCW]Contract (Effective Date)']= pd.to_datetime(df['[PCW]Contract (Effective Date)'])
df['[PCW]Start Date (Date)']= pd.to_datetime(df['[PCW]Start Date (Date)'])
df['[PCW]End Date (Date)']= pd.to_datetime(df['[PCW]End Date (Date)'])

In [ ]:
# create the df of contracts expiring within 365 days of today

from datetime import timedelta

one_year = np.timedelta64(365,'D')
today = np.datetime64('today')
mask = df['[PCW]Contract (Expiration Date)'] <= today + one_year 

df3 = df.loc[mask]

print(len(df3))
df3.head()

In [ ]:
# combine df2 and df3 into 1 report and exclude duplicates

contract_pipeline = pd.concat([df2,df3]).drop_duplicates().reset_index(drop=True)
print(len(contract_pipeline))
contract_pipeline.head()

In [ ]:
# read the LOB and Legal rep report into pandas

df_bu = pd.read_csv(r'filepath')
df_bu.columns

In [ ]:
# separate them into 2 dataframes

df_legal = df_bu[(df_bu['[PGP] Project Group Name'] == "Legal Team")]
df_biz = df_bu[(df_bu['[PGP] Project Group Name'] == "Line of Business")]

In [ ]:
# sometimes there are multiple legal and LOB reps to 1 contract
# it doesn't matter which rep is included in the report, so remove duplicates and tell python to just keep the first name

df_legal = df_legal.drop_duplicates(subset='[PGP] Project Id', keep='first')
df_biz = df_biz.drop_duplicates(subset='[PGP] Project Id', keep='first')

In [ ]:
# join df_legal to the contract_pipeline and assign to pipeline_report

#first, rename the project ID fields from all 3 dfs so that they match and can be looked up
df_legal = df_legal.rename(columns = {'[PGP] Project Id': 'Project Id', '[PGP] Member Name': 'Legal Rep'})
df_biz = df_biz.rename(columns = {'[PGP] Project Id': 'Project Id', '[PGP] Member Name': 'LOB Rep'})
contract_pipeline = contract_pipeline.rename(columns={'[PCW]Project (Project Id)': 'Project Id'})

In [ ]:
# merge the legal and LOB dfs to the pipeline report

from functools import reduce

dfs = [contract_pipeline, df_legal, df_biz]

pipeline_final = reduce(lambda left, right: pd.merge(left,right, on="Project Id"), dfs)
pipeline_final.head()

In [ ]:
# drop unnecessary columns
pipeline_final.drop(['[PGP] Project Group Name_x', '[PGP] Project Group Unique Name_x','count(ProjectGroup)_x',
                   '[PGP] Project Group Name_y', '[PGP] Project Group Unique Name_y', 'count(ProjectGroup)_y'],
                    axis=1,inplace=True)
pipeline_final.head()

In [ ]:
# drop duplicates from the pipeline_final and write to csv

pipeline_final = pipeline_final.drop_duplicates(subset='Project Id', keep='first')
pipeline_final.to_csv(r'filepath', index=False)